In [ ]:
%matplotlib inline
import sys
sys.path.append("..")

# Creating a video of bacteria

In this example we will create a video of moving bacteria.

## 1. Setup

Imports needed for this tutorial

In [ ]:
from deeptrack.scatterers import Ellipse
from deeptrack.optics import Fluorescence
from deeptrack.sequences import Sequence, Sequential, BrownianMotion
from deeptrack.noises import Poisson, Offset

from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## 2. Defining sequential features

For a feature's properties to evolve over time, we call it with the function Sequential. We additionally pass functions that define how properties over time as keyword arguments. These functions should output one value each time they are called and can optionally take some keyword arguments:

* sequence_step: The current position in the sequence
* sequence_length: The length of the sequence
* previous_value: The value of the property at the previous time step
* previous_values: A list of all previous values in the sequence.

If the property is already defined in the feature, then that property will be used for the first timestep.

Below we use this to image a rotating ellipse.

In [ ]:
optics = Fluorescence(
    NA=0.7,
    magnification=10,
    resolution=1e-6,
    wavelength=633e-9,
    output_region=(0, 0, 32, 32),
)

ellipse = Ellipse(
    position_unit="pixel",
    position=(16, 16),
    intensity=1,
    radius=(1e-6, 1.5e-6),
    rotation=0, # This will be the value at time 0.
)

def get_rotation(sequence_length=1, previous_value=0):
    return previous_value + 2*np.pi / sequence_length


rotating_ellipse = Sequential(ellipse, rotation=get_rotation)    

### Resolving videos

Sequential features (features that have been called with Sequential), can be used seamlessly with nonsequential features. To create the video, we first wrap the feature series with the feature Sequence, which defines the length of the sequence throught the property sequence_length.

In [ ]:
rotating_ellipse_sequence = Sequence(optics(ellipse), sequence_length=100)

video_of_rotating_ellipse = rotating_ellipse_sequence.resolve()

In [ ]:
# Function that takes a list of images and creates a playable video
def play(video):
    fig = plt.figure()

    images = []

    for image in video:
        images.append([plt.imshow(image[:, :, 0], cmap="gray")])

    ani = animation.ArtistAnimation(fig, images, interval=1/30 * 1000, blit=True,
                                    repeat_delay=1000)
    return HTML(ani.to_html5_video())

In [ ]:
play(video_of_rotating_ellipse)

## 3. Defining the bacteria

Bacteria will be approximated as ellipses. To do this we will use dummy properties, that is, properties that is not used by the feature to create the image. The names and values of these are completely arbitrary. These are

* velocity: the velocity of the bacteria
* area: the area of the ellipse
* rotation_diffusion: the standard distribution of the rotation, units rads/second

### Movement of the bacteria

The movement is defined by an angle and a velocity, both simulated as the previous value plus some normally distributed value. 

In [ ]:
# The velocity of the bacteria
def get_velocity(previous_value=1):
    return previous_value + np.random.randn() * 5e-7

# The direction of movement of the bacteria
def get_rotation(previous_value=0, rotation_diffusion=1, dt=1/30):
    return previous_value +  np.random.randn() * rotation_diffusion * dt

# The position of the cell.
def get_position(previous_value=(0, 0), rotation=0, velocity=0, dt=1/30):
    orientation = np.array((np.sin(rotation), np.cos(rotation)))
    return previous_value + (*(orientation * velocity * dt), np.random.randn()*1e-7)

### Shape of the bacteria

The bacteria is elongated along the direction of movement, proportionally to the velocity. The total area of the bacteria is conserved.

The intensity of the bacteria is

In [ ]:
# Primary and secondary radius of the bacteria
def get_radius(velocity=0, area=1e-12):
    ratio = np.min((1 + np.abs(velocity) / 4e-6, 3))
    b = np.sqrt(area / (ratio * np.pi))
    a = b * ratio
    return (a, b)

### Initial values of the properties

The initial values of the properties will be defined in the Ellipse feature. We have that,

* position is initialized as a random 2D position at a position between -256 and 512 for both x and y
* intensity is constant 1
* rotation is initialized as a random radian between 0 and 2pi
* rotation diffusion is a random number between 2 and 4.
* velocity is initialized as a normally distributed number with standard deviation 2e-6
* area is a random number between 5e-13 and 1e-12 (units m^2)


In [ ]:
bacteria = Ellipse(
    position=lambda: (-256 + np.random.rand(3) * 768) * 1e-7 * [1, 1, 0],
    intensity=1,
    rotation=lambda: np.random.rand() * 2 * np.pi,
    rotation_diffusion= lambda: np.random.rand() * 4 + 2,
    velocity=lambda: np.random.randn() * 2e-6,
    area=lambda:np.random.rand() * 5e-13 + 5e-13
)

### Making it sequential

In [ ]:
sequential_bacteria = Sequential(bacteria,
                                 velocity=get_velocity,
                                 rotation=get_rotation,
                                 position=get_position,
                                 radius=get_radius)

## 4. Defining the full feature series

We define the optics. Of note is that we lower the z resolution slightly to make the resolve time faster.

In [ ]:
optics = Fluorescence(
    magnification=10,
    resolution=(1e-6, 1e-6, 2e-6),
    wavelength=633e-9,
    output_region=(0, 0, 256, 256),
)

### Bringing it together

We define the full feature series as 100 bacteria imaged though the optical system, with a background offset and a Poisson noise, over 200 frames.

In [ ]:
imaged_bacteria = optics(sequential_bacteria**100) + Offset(offset=0.05) + Poisson(snr=60)
sequence_of_bacteria = Sequence(imaged_bacteria, sequence_length=200)  

In [ ]:
video_of_bacteria = sequence_of_bacteria.resolve()

play(video_of_bacteria)